In [1]:
import uproot3 as uproot
import uproot3_methods.classes.TLorentzVector as LVepm
import uproot3_methods.classes.TVector2 as LV2
import matplotlib.pyplot as plt
from matplotlib import gridspec
import infofile 
import numpy as np
import mplhep as hep
import awkward as ak

In [9]:
# Using data with truth values

sample_names = []
files = ["ttbar_reco_dilep_neutrinos.root"]

for file in files:
    sample_name = file.split(".")[0] # file is a string, splits it at ., and takes entry at index 1.
    print("Sample name: ", sample_name) 
    
    sample_names.append(sample_name)
    print("Sample names: ", sample_names)
        
    tree = uproot.open(file)["nominal"]
    
    #met_met, met_phi,  lep0_pt, lep0_eta, lep0_phi, lep0_e, lep1_pt, lep1_eta, lep1_phi, lep1_e, b0_pt, b0_eta, b0_phi, b0_e, b1_pt, b1_eta, b1_phi, b1_e= tree.arrays(["met_met", "met_phi", "lep0_pt", "lep0_eta", "lep0_phi", "lep0_e", "lep1_pt", "lep1_eta", "lep1_phi", "lep1_e", "b0_pt", "b0_eta", "b0_phi", "b0_e", "b1_pt", "b1_eta", "b1_phi", "b1_e"], outputtype=tuple)
    met_met, met_phi, lep0_pt, lep0_eta, lep0_phi, lep0_e, lep1_pt, lep1_eta, lep1_phi, lep1_e, b0_pt, b0_eta, b0_phi, b0_e, b1_pt, b1_eta, b1_phi, b1_e, v0_pt_truth, v0_eta_truth, v0_phi_truth, v0_m_truth, v1_pt_truth, v1_eta_truth, v1_phi_truth, v1_m_truth = tree.arrays(["met_met", "met_phi", "lep0_pt", "lep0_eta", "lep0_phi", "lep0_e", "lep1_pt", "lep1_eta", "lep1_phi", "lep1_e", "b0_pt", "b0_eta", "b0_phi", "b0_e", "b1_pt", "b1_eta", "b1_phi", "b1_e", "v0_pt_truth", "v0_eta_truth", "v0_phi_truth", "v0_m_truth", "v1_pt_truth", "v1_eta_truth", "v1_phi_truth", 'v1_m_truth'], outputtype=tuple)

    print("File has been successfully opened!")
    
    lep0 = LVepm.TLorentzVectorArray.from_ptetaphie(lep0_pt, lep0_eta, lep0_phi, lep0_e)
    lep1 = LVepm.TLorentzVectorArray.from_ptetaphie(lep1_pt, lep1_eta, lep1_phi, lep1_e)
    b0 = LVepm.TLorentzVectorArray.from_ptetaphie(b0_pt, b0_eta, b0_phi, b0_e)
    b1 = LVepm.TLorentzVectorArray.from_ptetaphie(b1_pt, b1_eta, b1_phi, b1_e)
    v0_truth = LVepm.TLorentzVectorArray.from_ptetaphim(v0_phi_truth, v0_eta_truth, v0_phi_truth, v0_m_truth)
    v1_truth = LVepm.TLorentzVectorArray.from_ptetaphim(v1_phi_truth, v1_eta_truth, v1_phi_truth, v1_m_truth)
    

system_array = [lep0, lep1, b0, b1, met_met, met_phi, v0_truth, v1_truth]
nEvents = len(lep0)
print(nEvents)

event = [lep0[0], lep1[0], b0[0], b1[0], met_met[0], met_phi[0], v0_truth[0], v1_truth[0]]
print('---------------------------------------------------')

print('Lep0: ', event[0])
print('Lep1: ', event[1])
print('b0: ', event[2])
print('b1: ', event[3])
print('met: ', event[4])
print('phi: ', event[5])

Sample name:  ttbar_reco_dilep_neutrinos
Sample names:  ['ttbar_reco_dilep_neutrinos']
File has been successfully opened!
10000
---------------------------------------------------
Lep0:  TLorentzVector(x=21576, y=15065, z=22534, t=34645)
Lep1:  TLorentzVector(x=35695, y=-52353, z=14250, t=64946)
b0:  TLorentzVector(x=41194, y=28352, z=78030, t=92974)
b1:  TLorentzVector(x=-18647, y=-39594, z=52083, t=68325)
met:  88773.09
phi:  2.5014758


In [3]:
# Classes and functions necessary functions
class Solution:
    def __init__(self):
        self.m_v1 = None
        self.m_v2 = None
        self.m_solutions = None
        
    def __init__(self, num):
        self.m_v1 = None
        self.m_v2 = None
        self.m_solutions = num
    
#    def __init__(self, v1, v2, num):
#        self.m_v1 = v1
#        self.m_v2 = v2
#        self.m_solutions = num

    def setSolutions1(self,num, a, b):
        self.m_v1=a
        self.m_v2=b
        self.m_solutions=num

    def setSolutions2(self,num):
        self.m_solutions=num

    def getNumSolutions(self):
        return self.m_solutions

    def getv1(self):
        return self.m_v1

    def getv2(self):
        return self.m_v2

In [3]:
def solveForNeutrinoEta(lepton, bJet, topmass, index, neutrinos):
    
    print("Eta = ", neutrinos[index][0])
    
    m_wmass = 80400
    Wmass2 = m_wmass * m_wmass
    m_bmass = 4180
    bmass = m_bmass

    # Perform a lorentz boost into the rest fram of the neutrino
    Elprime = lepton.E * neutrinos[index][2] - lepton.p3.z * neutrinos[index][1]
    #print('Elprime =', Elprime)
    Ebprime = bJet.E * neutrinos[index][2] - bJet.p3.z * neutrinos[index][1]
    #print('Ebprime =', Ebprime)

    A = (lepton.p3.y * Ebprime - bJet.p3.y * Elprime) / (bJet.p3.x * Elprime - lepton.p3.x * Ebprime)
    B = (Elprime * (topmass * topmass - Wmass2 - bmass * bmass - 2. *(lepton.p3.dot(bJet.p3) - lepton.E*bJet.E)) - Ebprime * Wmass2) /(2. * (lepton.p3.x * Ebprime - bJet.p3.x * Elprime))

    par1 = (lepton.p3.x * A + lepton.p3.y) / Elprime
    C = A * A + 1. - par1 * par1
    par2 = (Wmass2 / 2. + lepton.p3.x * B) / Elprime
    D = 2. * (A * B - par2 * par1)
    F = B * B - par2 * par2
    # Calculate discriminant
    det = D * D - 4. * C * F

    sol = Solution(0)

    if det>0:
    
        tmp = np.sqrt(det) / (2. * C)
        py1 = -D / (2. * C) + tmp
        py2 = -D / (2. * C) - tmp
        px1 = A * py1 + B
        px2 = A * py2 + B
        pT2_1 = px1 * px1 + py1 * py1
        pT2_2 = px2 * px2 + py2 * py2
        pz1 = np.sqrt(pT2_1) * neutrinos[index][1]
        pz2 = np.sqrt(pT2_2) * neutrinos[index][1]

        a1=LVepm.TLorentzVector.from_xyzm(px1, py1, pz1, 0)
        a2=LVepm.TLorentzVector.from_xyzm(px2, py2, pz2, 0)
        sol.setSolutions1(2, a1, a2);
        print(a1)
        print(a2)

    return sol

def neutrino_weight(neutrino1, neutrino2, met_ex, met_ey):
    # where do these sigma values come from?
    sigmax = 55643.82942235848
    sigmay = 54224.08180662581
    dx = met_ex - neutrino1.p3.x - neutrino2.p3.x
    dy = met_ey - neutrino1.p3.y - neutrino2.p3.y

    return np.exp(-dx * dx / (2. * sigmax * sigmax) - dy * dy / (2. * sigmay * sigmay))

In [10]:
# Sonnenschein's Method
[lep0, lep1, b0, b1, met_met, met_phi] = event[:6]
print(lep1.mag2)

'''# Mass
mW = 80400
mT = 172500
mB = b0.mass
mBbar = b1.mass
mL = lep0.mass
mLbar = lep1.mass
# Energy of leptons and b-jets
eB = b0.E
eL = lep0.E
eBbar = b1.E
eLbar = lep1.E

# 3-momenta of b-jets and leptons
pBx = b0.p3.x
pBy = b0.p3.y
pBz = b0.p3.z

pBbarx = b1.p3.x
pBbary = b1.p3.y
pBbarz = b.p3.z

pLx = lep0.p3.x
pLy = lep0.p3.y
pLz = lep0.p3.z

pLbarx = lep1.p3.x
pLbary = lep1.p3.y
pLbarz = lep1.p3.z

# Coefficients
#a
a1 = (eB + eL)*(mW**2 - mL**2) - eL*(mT**2-mB**2-mL**2)
a2 = 2*(eB*pLx - eL*pBx)
a3 = 2*(eB*pLy - eL*pBy)
a4 = 2*(eB*pLz - eL*pBz)

#b
b1 = (eBbar + eLbar)*(mW**2 - mLbar**2) - eLbar*(mT**2-mBbar**2-mLbar**2)
b2 = 2*(eBbar*pLbarx - eLbar*pBbarx)
b3 = 2*(eBbar*pLbary - eLbar*pBbary)
b4 = 2*(eBbar*pLbarz - eLbar*pBbarz)

#c
c00 = -4*(eL**2 - pLy**2) - 4*(eL**2 - pLz**2)*(a3/a4)**2 - 8*pLy*pLz*(a3/a4)
c10 = -8*(eL**2 - pLz**2)*a2*a3/(a4**2) + 8*pLx*pLy - 8*pLx*pLz*(a3/a4) - 8*pLy*pLz*(a2/a4)
c11 = 4*(mW**2 - mL**2)*(pLy-pLz*(a3/a4)) - 8*(eL**2 - pLz**2)*(a1*a3/a4**2) - 8*pLy*pLz*(a1/a4)
c20 = -4*(eL**2 - pLx**2) - 4*(eL**2 - pLz**2)*(a2/a4)**2 - 8*pLx*pLz*(a2/a4)
c21 = 4*(mW**2 -mL**2)*(pLx - pLz*a2/a4) - 8*(eL**2 - pLz**2)*(a1*a2/a4**2) - 8*pLx*pLz*(a1/a4)
c22 = (mW**2 - mL**2)**2 - 4*(eL**2 - pLz**2)*(a1/a4)**2 - 4*(mW**2 - mL**2)*pLz*(a1/a4)

#dprime
dprime00 = -4*(eLbar**2 - pLbary**2) - 4*(eLbar**2 - pLbarz**2)*(a3/a4)**2 - 8*pLbary*pLbarz*(b3/b4)
dprime10 = -8*(eLbar**2 - pLbarz**2)*b2*b3/(b4**2) + 8*pLbarx*pLbary - 8*pLbarx*pLbarz*(b3/b4) - 8*pLbary*pLbarz*(b2/b4)
dprime11 = 4*(mW**2 - mLbar**2)*(pLbary-pLbarz*(b3/b4)) - 8*(eLbar**2 - pLbarz**2)*(b1*b3/b4**2) - 8*pLbary*pLbarz*(b1/b4)
dprime20 = -4*(eLbar**2 - pLbarx**2) - 4*(eLbar**2 - pLbarz**2)*(b2/b4)**2 - 8*pLbarx*pLbarz*(b2/b4)
dprime21 = 4*(mW**2 -mLbar**2)*(pLbarx - pLbarz*b2/b4) - 8*(eLbar**2 - pLbarz**2)*(b1*b2/b4**2) - 8*pLbarx*pLbarz*(b1/b4)
dprime22 = (mW**2 - mLbar**2)**2 - 4*(eLbar**2 - pLbarz**2)*(b1/b4)**2 - 4*(mW**2 - mLbar**2)*pLbarz*(b1/b4)

# Missing transverse energy
metx = met_met*np.cos(met_phi)
mety = met_met*np.sin(met_phi)

#d
d00 = dprime00
d10 = dprime10
d11 = -dprime11 - 2*mety*dprime00 - metx*dprime10
d20 = dprime20
d21 = -dprime21 - 2*metx*dprime20 - mety*dprime10
d22 = dprime22 + (metx**2)*dprime20 + (mety**2)*dprime00 + metx*mety*dprime10 + metx*dprime21 + mety*dprime11

#h
h4 = (c00**2)*(d22**2) + c11*d22*(c11*d00 - c00*d11) + c00*c22*(d11**2 - 2*d00*d22) - c22*d00*(c22*d00 - c11*d11)
h3 = c00*d21*(2*c00*d22 - c11*d11) + c00*d11*(2*c22*d10 + c21*d11) + c22*d00*(2*c21*d00 - c11*d10) - c00*d22(c11*d10 + c10*d11) - 2*c00*d00*(c22*d21 +c21*d22) - d00*d11*(c11*c21 + c10*c22) + c11*d00*(c11*d21 + 2*c10*d22)
h2 = (c00**2)*(2*d22*d20 + d21**2) - c00*d21*(c11*d10 + c10*d11) + c11*d20*(c11*d00 - c00*d11) + c00*d10*(c22*d10 - c10*d22) + c00*d11*(2*c21*d10 + c20*d11) + (2*c22*c20 + c21**2)*(d00**2) - 2*c00*d00*(c22*d20 + c21*d21 + c20*d22) + c10*d00*(2*c11*d21 + c10*d22) - d00*d10*(c11*c21 + c10*c22) - d00*d11*(c11*c20 + c10*c21)
h1 = c00*d21*(2*c00*d20 - c10*d10) - c00*d20*(c11*d10 + c10*d11) + c00*d10*(c21*d10 + 2*c20*d11) - 2*c00*d00*(c21*d20 + c20*d21) + c10*d00*(2*c11*d20 + c10*d21) - c20*d00*(2*c21*d00 - c10*d11) - d00*d11*(c11*c20 + c10*c21)
h0 = (c00**2)*(d20**2) + c10*d20*(c10*d00 - c00*d10) + c20*d10*(c00*d10 - c10*d00) + c20*d00*(c20*d00 - 2*c00*d20)

possiblePx = np.roots([h0, h1, h2, h3, h4])
print(possiblePx)
'''

-145.59839969873428


'# Mass\nmW = 80400\nmT = 172500\nmB = b0.mass\nmBbar = b1.mass\nmL = lep0.mass\nmLbar = lep1.mass\n# Energy of leptons and b-jets\neB = b0.E\neL = lep0.E\neBbar = b1.E\neLbar = lep1.E\n\n# 3-momenta of b-jets and leptons\npBx = b0.p3.x\npBy = b0.p3.y\npBz = b0.p3.z\n\npBbarx = b1.p3.x\npBbary = b1.p3.y\npBbarz = b.p3.z\n\npLx = lep0.p3.x\npLy = lep0.p3.y\npLz = lep0.p3.z\n\npLbarx = lep1.p3.x\npLbary = lep1.p3.y\npLbarz = lep1.p3.z\n\n# Coefficients\n#a\na1 = (eB + eL)*(mW**2 - mL**2) - eL*(mT**2-mB**2-mL**2)\na2 = 2*(eB*pLx - eL*pBx)\na3 = 2*(eB*pLy - eL*pBy)\na4 = 2*(eB*pLz - eL*pBz)\n\n#b\nb1 = (eBbar + eLbar)*(mW**2 - mLbar**2) - eLbar*(mT**2-mBbar**2-mLbar**2)\nb2 = 2*(eBbar*pLbarx - eLbar*pBbarx)\nb3 = 2*(eBbar*pLbary - eLbar*pBbary)\nb4 = 2*(eBbar*pLbarz - eLbar*pBbarz)\n\n#c\nc00 = -4*(eL**2 - pLy**2) - 4*(eL**2 - pLz**2)*(a3/a4)**2 - 8*pLy*pLz*(a3/a4)\nc10 = -8*(eL**2 - pLz**2)*a2*a3/(a4**2) + 8*pLx*pLy - 8*pLx*pLz*(a3/a4) - 8*pLy*pLz*(a2/a4)\nc11 = 4*(mW**2 - mL**2)*(pLy-pLz

In [4]:
# Create neutrino array
neutrinos = np.zeros((2000,3))
etaStep = 0.1
index = 0 
eta=-1

while eta<1.0001:
    neutrinos[index][0]=eta
    neutrinos[index][1]=np.sinh(eta)
    neutrinos[index][2]=np.cosh(eta)
    index +=1
    eta+=etaStep
    
etaSize=index
#nEvents = num_events_cut3[0]
nEvents =1

# Arrays to hold neutrino data
v0_pt = np.zeros(nEvents)
v0_eta = np.zeros(nEvents)
v0_phi = np.zeros(nEvents)
v0_m = np.zeros(nEvents)
v0_e = np.zeros(nEvents)
v1_pt = np.zeros(nEvents)
v1_eta = np.zeros(nEvents)
v1_phi = np.zeros(nEvents)
v1_m = np.zeros(nEvents)
v1_e = np.zeros(nEvents)

# Lepton and Jet 4-Vectors
l0 = event[0]
l1 = event[1]
b0 = event[2]
b1 = event[3]

# We need MET in x- and y-directions, currently in polar coordinates
met_ex = event[4]*np.cos(event[5])
met_ey = event[4]*np.sin(event[5])

max_weight=0
topMass=172500
weight=0

# Attempt to solve eqns for different values of eta
for j in range(0, etaSize):

    # Need to define solveForNeutrinoEta
    print('Answer 1')
    ans1= solveForNeutrinoEta(l0, b0, topMass, j, neutrinos)
    print('Answer 2')
    ans2= solveForNeutrinoEta(l1, b1, topMass, j, neutrinos)
    print('Answer 3')
    ans3= solveForNeutrinoEta(l0, b1, topMass, j, neutrinos)
    print('Answer 4')
    ans4= solveForNeutrinoEta(l1, b0, topMass, j, neutrinos)

    if (ans1.getNumSolutions()>0 and ans2.getNumSolutions()>0):
        sol1 = neutrino_weight(ans1.getv1(), ans2.getv1(), met_ex, met_ey)
        print("Weight 1: ", sol1)
        sol2 = neutrino_weight(ans1.getv1(), ans2.getv2(), met_ex, met_ey)
        print("Weight 2: ", sol2)
        sol3 = neutrino_weight(ans1.getv2(), ans2.getv1(), met_ex, met_ey)
        print("Weight 3: ", sol3)
        sol4 = neutrino_weight(ans1.getv2(), ans2.getv2(), met_ex, met_ey)
        print("Weight 4: ", sol4)

    if (ans3.getNumSolutions()>0 and ans4.getNumSolutions()>0):
        sol5 = neutrino_weight(ans3.getv1(), ans4.getv1(), met_ex, met_ey)
        print("Weight 5: ", sol5)
        sol6 = neutrino_weight(ans3.getv1(), ans4.getv2(), met_ex, met_ey)
        print("Weight 6: ", sol6)
        sol7 = neutrino_weight(ans3.getv2(), ans4.getv1(), met_ex, met_ey)
        print("Weight 7: ", sol7)
        sol8 = neutrino_weight(ans3.getv2(), ans4.getv2(), met_ex, met_ey)
        print("Weight 8: ", sol8)

    # Save the solution with the highest weight
    if (ans1.getNumSolutions() > 0 and ans2.getNumSolutions()>0):
        if (sol1>weight):
            weight=sol1
            v0 = ans1.getv1()
            v2 = ans2.getv2()

        if (sol2 > weight):
            weight = sol2
            v0 = ans1.getv1()
            v1 = ans2.getv2()

        if (sol3 > weight):
            weight = sol3
            v0 = ans1.getv2()
            v1 = ans2.getv1()

        if (sol4 > weight):
            weight = sol4
            v0 = ans2.getv2()
            v1 = ans2.getv2()

    if (ans3.getNumSolutions() > 0 and ans4.getNumSolutions() > 0):
        if (sol5 > weight):
            weight = sol5
            v0 = ans3.getv1()
            v1 = ans4.getv1()

        if (sol6 > weight):
            weight = sol6
            v0 = ans3.getv1()
            v1 = ans4.getv2()

        if (sol7 > weight):
            weight = sol7
            v0 = ans3.getv2()
            v1 = ans4.getv1()

        if (sol8 > weight):
            weight = sol8
            v0 = ans3.getv2()
            v1 = ans4.getv2()

    #Save solution with largest weight per neutrino eta
    if (weight > max_weight):
        max_weight = weight
        bv0 = v0
        bv1 = v1

# Save solution to neutrino array
i=0
if (bv0 is not None) and (bv1 is not None):
    v0_pt[i] = bv0.pt
    v0_eta[i] = bv0.eta
    v0_phi[i] = bv0.phi
    v0_e[i] = bv0.E
    v0_m[i] = 0
    v1_pt[i] = bv1.pt
    v1_eta[i] = bv1.eta
    v1_phi[i] = bv1.phi
    v1_m[i] = 0
    v1_e[i] = bv1.E

Answer 1
Eta =  -1.0
Answer 2
Eta =  -1.0
Answer 3
Eta =  -1.0
Answer 4
Eta =  -1.0
Answer 1
Eta =  -0.9
Answer 2
Eta =  -0.9
Answer 3
Eta =  -0.9
Answer 4
Eta =  -0.9
Answer 1
Eta =  -0.8
Answer 2
Eta =  -0.8
Answer 3
Eta =  -0.8
TLorentzVector(x=39179, y=65505, z=-67787, t=1.0208e+05)
TLorentzVector(x=71381, y=36685, z=-71276, t=1.0734e+05)
Answer 4
Eta =  -0.8
TLorentzVector(x=58176, y=-66380, z=-78389, t=1.1805e+05)
TLorentzVector(x=13229, y=-76134, z=-68629, t=1.0335e+05)
Weight 5:  0.00620838667715477
Weight 6:  0.042584018828877024
Weight 7:  0.0004657276592455355
Weight 8:  0.004633304152002831
Answer 1
Eta =  -0.7000000000000001
TLorentzVector(x=63177, y=67887, z=-70348, t=1.164e+05)
TLorentzVector(x=85153, y=37250, z=-70506, t=1.1666e+05)
Answer 2
Eta =  -0.7000000000000001
Answer 3
Eta =  -0.7000000000000001
TLorentzVector(x=26066, y=77704, z=-62173, t=1.0287e+05)
TLorentzVector(x=87112, y=23114, z=-68369, t=1.1312e+05)
Answer 4
Eta =  -0.7000000000000001
TLorentzVector(x=79

In [5]:
v0_reco_xyz = LVepm.TLorentzVector.from_ptetaphi(v0_pt[0], v0_eta[0], v0_phi[0], v0_e[0])
v0_reco = LVepm.PtEtaPhiMassLorentzVector(v0_reco_xyz.pt, v0_reco_xyz.eta, v0_reco_xyz.phi, 0)
v1_reco_xyz = LVepm.TLorentzVector.from_ptetaphi(v1_pt[0], v1_eta[0], v1_phi[0], v1_e[0])
v1_reco = LVepm.PtEtaPhiMassLorentzVector(v1_reco_xyz.pt, v1_reco_xyz.eta, v1_reco_xyz.phi, 0)

In [6]:
print('v0 Reco: ', v0_reco)
print('v0 Truth: ', v0_truth[0])

print('v1 Reco: ', v1_reco)
print('v1 Truth: ', v1_truth[0])

v0 Reco:  PtEtaPhiMassLorentzVector(pt=1.3372e+05, eta=-1.3878e-16, phi=1.7709, mass=0)
v0 Truth:  PtEtaPhiMassLorentzVector(pt=-1.6359, eta=3.0414, phi=-1.6359, mass=52.107)
v1 Reco:  PtEtaPhiMassLorentzVector(pt=1.2516e+05, eta=-1.3878e-16, phi=-1.8777, mass=0)
v1 Truth:  PtEtaPhiMassLorentzVector(pt=-0.89597, eta=2.7591, phi=-0.89597, mass=7.2474)
